In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [18]:
keyword = "Canon RF 200-800mm"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

data = []

for page_number in range(1, 6):
    url = f"https://www.ebay.com/sch/i.html?_nkw={keyword}&_pgn={page_number}&_ipg=240"
    result = requests.get(url, headers=headers)
    if result.status_code == 200:
        soup = BeautifulSoup(result.content, 'html.parser')
        listings = soup.find_all('li', attrs={'class': 's-item'})
        
        for listing in listings:
            # Dictionary to store the data for each listing
            listing_dict = {}
        
            title = listing.find('div', attrs={'class': 's-item__title'})
            if title:
                listing_dict['Title'] = title.get_text()
        
            price = listing.find('span', attrs={'class': 's-item__price'})
            if price:
                listing_dict['price'] = price.get_text()
        
            url = listing.find('a', attrs={'class': 's-item__link'})
            if url:
                listing_dict['url'] = url['href']
        
            shipping = listing.find('span', attrs={'class': "s-item__shipping s-item__logisticsCost"})
            if shipping:
                listing_dict['shipping'] = shipping.get_text()
        
            location = listing.find('span', attrs={'class': 's-item__location s-item__itemLocation'})
            if location:
                listing_dict['location'] = location.get_text()
        
            condition = listing.find('span', attrs={'class': 'SECONDARY_INFO'})
            if condition:
                listing_dict['condition'] = condition.get_text()
        
            listing_type = listing.find('span', attrs={'class':'s-item__purchase-options s-item__purchaseOptions'})
            if listing_type:
                listing_dict['listing_type'] = listing_type.get_text()
            data.append(listing_dict)
        time.sleep(1)
        
# Create DataFrame
df = pd.DataFrame(data)

df.to_csv(f"{keyword} ebay-list.csv", index=False)